In [1]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
# from rsna_retro.train3d import *
from rsna_retro.train_adjacent import *

Loading imports


In [2]:
torch.cuda.set_device(2)

## Train

In [3]:
name = 'train_adjacent_5c'

In [4]:
dbunch = get_adj_data(512, 128, Meta.splits_stg1)
learn = get_learner(dbunch, partial(xresnet18, c_in=5))

In [5]:
do_fit(learn, 20, 4e-2)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.259508,0.264516,0.898018,0.824924,06:03
1,0.231305,0.254917,0.910475,0.848391,06:01
2,0.211863,0.215509,0.921635,0.862720,06:02
3,0.203018,0.211351,0.923065,0.867408,06:00
4,0.199288,0.235727,0.917799,0.858694,06:05
5,0.190459,0.219281,0.922430,0.858403,06:06
6,0.188275,0.215355,0.923750,0.868547,06:07
7,0.183203,0.218013,0.927466,0.865713,06:05
8,0.179288,0.188682,0.932985,0.881552,06:04
9,0.176608,0.190069,0.932234,0.880387,06:01


In [6]:
learn.dbunch = get_data_gen(Meta.df_any, Meta.grps_any, 512, sz=256)
do_fit(learn, 20, 4e-3)
learn.save(f'runs/{name}-2')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.170900,0.182028,0.935947,0.886717,07:02
1,0.157297,0.171350,0.939544,0.893312,06:55
2,0.149589,0.167156,0.940802,0.897126,06:56
3,0.146365,0.166136,0.941752,0.898159,06:53
4,0.143014,0.164770,0.941443,0.895034,06:56
5,0.141222,0.166002,0.942246,0.897047,06:53
6,0.136026,0.164642,0.943054,0.898265,06:52
7,0.133535,0.165213,0.942979,0.897815,06:52
8,0.132073,0.164068,0.943747,0.899272,06:52
9,0.125835,0.163462,0.943464,0.900517,06:52


In [7]:
learn.dbunch = get_data_gen(Meta.df_comb, Meta.grps, 512, sz=None)
do_fit(learn, 2, 5e-4)
learn.save(f'runs/{name}-3')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.093190,0.190156,0.943804,0.899589,06:46
1,0.085082,0.192222,0.943597,0.898954,06:41


## Submission

In [5]:
learn.load(f'runs/{name}-3')
sub_fn = f'subm/{name}'

In [6]:
# learn.dbunch = get_data_gen(Meta.df_tst, bs=512, sz=None, with_aug=False, tst_dir='tst_jpg256')
learn.dbunch = get_adj_test_data(512, sz=None, tst_dir='tst_jpg256')

In [7]:
preds,targs = learn.get_preds()

In [8]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [11]:
api.competition_submit(f'{sub_fn}.csv', f'{name} adjacent 5c', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 9.80MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [16]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14244524,
 'totalBytes': 27277209,
 'date': '2020-01-21T07:07:06.64Z',
 'description': 'train_adjacent_5c adjacent 5c',
 'errorDescription': None,
 'fileName': 'train_adjacent_5c.csv',
 'publicScore': '0.34569',
 'privateScore': '0.07648',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14244524/14244524.raw'}